In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import datetime
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['font.serif'] = ['SimHei']
import seaborn as sns
%matplotlib inline
p = sns.color_palette()
sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})

#https://github.com/mwaskom/seaborn/issues/1009

import os
print (os.path.abspath(os.curdir))

/home/fg/IdeaProjects/jd/Untitled Folder


In [2]:
t_order="../input/t_order.csv"

In [3]:
order_info = pd.read_csv(t_order,low_memory=False)
#数据清洗
#过滤总价<打折价的
order_info = order_info.query('price * qty >= discount')
#过滤掉单价为0的
order_info=order_info[order_info['price']!=0]
order_info['uid'] = order_info['uid'].astype('uint32')
order_info['buy_time'] = order_info['buy_time'].astype('datetime64[ns]')
order_info['qty'] = order_info['qty'].astype('uint32')
order_info['cate_id'] = order_info['cate_id'].astype('str')
order_info['cost']=order_info['price']*order_info['qty']-order_info['discount']
order_info.drop(['qty','price','discount'],1,inplace=True)
order_info['month'] = pd.DatetimeIndex(order_info.buy_time).month
order_info.drop(['buy_time'],1,inplace=True)
order_info = pd.get_dummies(order_info)
order_info.info()
order_info.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5390625 entries, 0 to 5400777
Data columns (total 47 columns):
uid           uint32
cost          float64
month         int64
cate_id_1     uint8
cate_id_10    uint8
cate_id_11    uint8
cate_id_12    uint8
cate_id_13    uint8
cate_id_14    uint8
cate_id_15    uint8
cate_id_16    uint8
cate_id_17    uint8
cate_id_18    uint8
cate_id_19    uint8
cate_id_2     uint8
cate_id_20    uint8
cate_id_21    uint8
cate_id_22    uint8
cate_id_23    uint8
cate_id_24    uint8
cate_id_25    uint8
cate_id_26    uint8
cate_id_27    uint8
cate_id_28    uint8
cate_id_29    uint8
cate_id_3     uint8
cate_id_30    uint8
cate_id_31    uint8
cate_id_32    uint8
cate_id_33    uint8
cate_id_34    uint8
cate_id_35    uint8
cate_id_36    uint8
cate_id_37    uint8
cate_id_38    uint8
cate_id_39    uint8
cate_id_4     uint8
cate_id_40    uint8
cate_id_41    uint8
cate_id_42    uint8
cate_id_43    uint8
cate_id_44    uint8
cate_id_5     uint8
cate_id_6     uint8
cate

,uid,cost,month,cate_id_1,cate_id_10,cate_id_11,cate_id_12,cate_id_13,cate_id_14,cate_id_15,...,cate_id_40,cate_id_41,cate_id_42,cate_id_43,cate_id_44,cate_id_5,cate_id_6,cate_id_7,cate_id_8,cate_id_9
0,45370,3.995009,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,66975,3.269410,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,75358,2.255235,11,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
order_info_08M=order_info[order_info['month']==8]
order_info_09M=order_info[order_info['month']==9]
order_info_10M=order_info[order_info['month']==10]
order_info_11M=order_info[order_info['month']==11]

In [5]:
order_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5390625 entries, 0 to 5400777
Data columns (total 4 columns):
uid        uint32
cate_id    object
cost       float64
month      int64
dtypes: float64(1), int64(1), object(1), uint32(1)
memory usage: 185.1+ MB


In [5]:
def feature_sum_count_max_min_mean_median_std_var( dataset, month):
    d=dataset
    gb= d.loc[:,['uid','cost']].groupby('uid',as_index=False)

    cost_sum=gb.sum()
    cost_sum.columns=['uid', str(month)+'M_cost_sum']
    feature=cost_sum

    cost_count=gb.count()
    cost_count.columns=['uid', str(month)+'M_cost_count']
    feature=pd.merge(feature, cost_count,how='left', on = 'uid')

    cost_max=gb.max()
    cost_max.columns=['uid', str(month)+'M_cost_max' ]
    feature=pd.merge(feature, cost_max,how='left', on = 'uid')

    cost_min=gb.min()
    cost_min.columns=['uid', str(month)+'M_cost_min']
    feature=pd.merge(feature, cost_min,how='left', on = 'uid')

    cost_mean=gb.mean()
    cost_mean.columns=['uid', str(month)+'M_cost_mean']
    feature=pd.merge(feature, cost_mean,how='left', on = 'uid')

    return feature

In [6]:
order_info_summary=order_info.loc[:,['uid']].drop_duplicates()
feature_08M_summary=feature_sum_count_max_min_mean_median_std_var(order_info_08M, 8)
order_info_summary=pd.merge(order_info_summary, feature_08M_summary,how='left', on = 'uid')
feature_09M_summary=feature_sum_count_max_min_mean_median_std_var(order_info_09M, 9)
order_info_summary=pd.merge(order_info_summary, feature_09M_summary,how='left', on = 'uid')
feature_10M_summary=feature_sum_count_max_min_mean_median_std_var(order_info_10M, 10)
order_info_summary=pd.merge(order_info_summary, feature_10M_summary,how='left', on = 'uid')
feature_11M_summary=feature_sum_count_max_min_mean_median_std_var(order_info_11M, 11)
order_info_summary=pd.merge(order_info_summary, feature_11M_summary,how='left', on = 'uid')
order_info_summary.fillna(0,inplace=True)
order_info_summary.fillna(0.0,inplace=True)

In [7]:
order_info_summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89240 entries, 0 to 89239
Data columns (total 21 columns):
uid               89240 non-null uint32
8M_cost_sum       89240 non-null float64
8M_cost_count     89240 non-null float64
8M_cost_max       89240 non-null float64
8M_cost_min       89240 non-null float64
8M_cost_mean      89240 non-null float64
9M_cost_sum       89240 non-null float64
9M_cost_count     89240 non-null float64
9M_cost_max       89240 non-null float64
9M_cost_min       89240 non-null float64
9M_cost_mean      89240 non-null float64
10M_cost_sum      89240 non-null float64
10M_cost_count    89240 non-null float64
10M_cost_max      89240 non-null float64
10M_cost_min      89240 non-null float64
10M_cost_mean     89240 non-null float64
11M_cost_sum      89240 non-null float64
11M_cost_count    89240 non-null float64
11M_cost_max      89240 non-null float64
11M_cost_min      89240 non-null float64
11M_cost_mean     89240 non-null float64
dtypes: float64(20), uint32(1)


In [8]:
order_info_summary['9M_8M_cost']=order_info_summary['9M_cost_sum']-order_info_summary['8M_cost_sum']
order_info_summary['10M_9M_cost']=order_info_summary['10M_cost_sum']-order_info_summary['9M_cost_sum']
order_info_summary['11M_10M_cost']=order_info_summary['11M_cost_sum']-order_info_summary['10M_cost_sum']

In [9]:
order_info_summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89240 entries, 0 to 89239
Data columns (total 24 columns):
uid               89240 non-null uint32
8M_cost_sum       89240 non-null float64
8M_cost_count     89240 non-null float64
8M_cost_max       89240 non-null float64
8M_cost_min       89240 non-null float64
8M_cost_mean      89240 non-null float64
9M_cost_sum       89240 non-null float64
9M_cost_count     89240 non-null float64
9M_cost_max       89240 non-null float64
9M_cost_min       89240 non-null float64
9M_cost_mean      89240 non-null float64
10M_cost_sum      89240 non-null float64
10M_cost_count    89240 non-null float64
10M_cost_max      89240 non-null float64
10M_cost_min      89240 non-null float64
10M_cost_mean     89240 non-null float64
11M_cost_sum      89240 non-null float64
11M_cost_count    89240 non-null float64
11M_cost_max      89240 non-null float64
11M_cost_min      89240 non-null float64
11M_cost_mean     89240 non-null float64
9M_8M_cost        89240 non-nul

In [10]:
def feature_cate_id_sum( dataset, month ):
    d=dataset
    gb= d.loc[:,['uid', 'cate_id_1', 'cate_id_10', 'cate_id_11', 'cate_id_12', 'cate_id_13', 'cate_id_14', 'cate_id_15', 'cate_id_16', 'cate_id_17', 'cate_id_18', 'cate_id_19', 'cate_id_2', 'cate_id_20', 'cate_id_21', 'cate_id_22', 'cate_id_23', 'cate_id_24', 'cate_id_25', 'cate_id_26', 'cate_id_27', 'cate_id_28', 'cate_id_29', 'cate_id_3', 'cate_id_30', 'cate_id_31', 'cate_id_32', 'cate_id_33', 'cate_id_34', 'cate_id_35', 'cate_id_36', 'cate_id_37', 'cate_id_38', 'cate_id_39', 'cate_id_4', 'cate_id_40', 'cate_id_41', 'cate_id_42', 'cate_id_43', 'cate_id_44', 'cate_id_5', 'cate_id_6', 'cate_id_7', 'cate_id_8', 'cate_id_9']].groupby('uid',as_index=False)
    cate_id_sum=gb.sum()
    cate_id_sum.columns=['uid', str(month)+'M_cate_id_1', str(month)+'M_cate_id_10', str(month)+'M_cate_id_11', str(month)+'M_cate_id_12', str(month)+'M_cate_id_13', str(month)+'M_cate_id_14', str(month)+'M_cate_id_15', str(month)+'M_cate_id_16', str(month)+'M_cate_id_17', str(month)+'M_cate_id_18', str(month)+'M_cate_id_19', str(month)+'M_cate_id_2', str(month)+'M_cate_id_20', str(month)+'M_cate_id_21', str(month)+'M_cate_id_22', str(month)+'M_cate_id_23', str(month)+'M_cate_id_24', str(month)+'M_cate_id_25', str(month)+'M_cate_id_26', str(month)+'M_cate_id_27', str(month)+'M_cate_id_28', str(month)+'M_cate_id_29', str(month)+'M_cate_id_3', str(month)+'M_cate_id_30', str(month)+'M_cate_id_31', str(month)+'M_cate_id_32', str(month)+'M_cate_id_33', str(month)+'M_cate_id_34', str(month)+'M_cate_id_35', str(month)+'M_cate_id_36', str(month)+'M_cate_id_37', str(month)+'M_cate_id_38', str(month)+'M_cate_id_39', str(month)+'M_cate_id_4', str(month)+'M_cate_id_40', str(month)+'M_cate_id_41', str(month)+'M_cate_id_42', str(month)+'M_cate_id_43', str(month)+'M_cate_id_44', str(month)+'M_cate_id_5', str(month)+'M_cate_id_6', str(month)+'M_cate_id_7', str(month)+'M_cate_id_8', str(month)+'M_cate_id_9' ]
    feature=cate_id_sum
    return feature

In [11]:
feature_08M_dummies_cate_id=feature_cate_id_sum(order_info_08M, 8)
order_info_summary=pd.merge(order_info_summary, feature_08M_dummies_cate_id,how='left', on = 'uid')
feature_09M_dummies_cate_id=feature_cate_id_sum(order_info_09M, 9)
order_info_summary=pd.merge(order_info_summary, feature_09M_dummies_cate_id,how='left', on = 'uid')
feature_10M_dummies_cate_id=feature_cate_id_sum(order_info_10M, 10)
order_info_summary=pd.merge(order_info_summary, feature_10M_dummies_cate_id,how='left', on = 'uid')
feature_11M_dummies_cate_id=feature_cate_id_sum(order_info_11M, 11)
order_info_summary=pd.merge(order_info_summary, feature_11M_dummies_cate_id,how='left', on = 'uid')

In [12]:
print(order_info_summary.columns.values)

['uid' '8M_cost_sum' '8M_cost_count' '8M_cost_max' '8M_cost_min'
 '8M_cost_mean' '9M_cost_sum' '9M_cost_count' '9M_cost_max' '9M_cost_min'
 '9M_cost_mean' '10M_cost_sum' '10M_cost_count' '10M_cost_max'
 '10M_cost_min' '10M_cost_mean' '11M_cost_sum' '11M_cost_count'
 '11M_cost_max' '11M_cost_min' '11M_cost_mean' '9M_8M_cost' '10M_9M_cost'
 '11M_10M_cost' '8M_cate_id_1' '8M_cate_id_10' '8M_cate_id_11'
 '8M_cate_id_12' '8M_cate_id_13' '8M_cate_id_14' '8M_cate_id_15'
 '8M_cate_id_16' '8M_cate_id_17' '8M_cate_id_18' '8M_cate_id_19'
 '8M_cate_id_2' '8M_cate_id_20' '8M_cate_id_21' '8M_cate_id_22'
 '8M_cate_id_23' '8M_cate_id_24' '8M_cate_id_25' '8M_cate_id_26'
 '8M_cate_id_27' '8M_cate_id_28' '8M_cate_id_29' '8M_cate_id_3'
 '8M_cate_id_30' '8M_cate_id_31' '8M_cate_id_32' '8M_cate_id_33'
 '8M_cate_id_34' '8M_cate_id_35' '8M_cate_id_36' '8M_cate_id_37'
 '8M_cate_id_38' '8M_cate_id_39' '8M_cate_id_4' '8M_cate_id_40'
 '8M_cate_id_41' '8M_cate_id_42' '8M_cate_id_43' '8M_cate_id_44'
 '8M_cate_id_5'

In [14]:
order_info_summary.info()
order_info_summary.columns
order_info_summary

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89240 entries, 0 to 89239
Columns: 200 entries, uid to 11cate_id_9
dtypes: float64(199), uint32(1)
memory usage: 136.5 MB


,uid,8M_cost_sum,8M_cost_count,8M_cost_max,8M_cost_min,8M_cost_mean,9M_cost_sum,9M_cost_count,9M_cost_max,9M_cost_min,...,11cate_id_40,11cate_id_41,11cate_id_42,11cate_id_43,11cate_id_44,11cate_id_5,11cate_id_6,11cate_id_7,11cate_id_8,11cate_id_9
0,45370,740.353623,167.0,31.179925,0.003114,4.433255,194.677788,69.0,12.776645,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,66975,295.558678,97.0,8.139054,0.017096,3.046997,153.932602,50.0,6.140830,0.430677,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,75358,216.595801,7.0,112.521285,0.708680,30.942257,80.384712,3.0,80.372347,0.006182,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,40597,109.410117,64.0,4.909172,0.096124,1.709533,13.316281,8.0,3.070415,0.214459,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,83886,45.014067,14.0,6.621853,0.668963,3.215291,10.012002,5.0,2.795889,0.931437,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,63459,143.457470,80.0,12.527512,0.129749,1.793218,66.116206,23.0,15.335165,0.170247,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,72200,70.139694,40.0,6.457724,0.220728,1.753492,154.468503,74.0,6.453131,0.053037,...,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
7,48365,80.103167,22.0,10.223688,0.323900,3.641053,70.154966,26.0,5.763408,0.534880,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,62038,1.861502,2.0,1.831038,0.030464,0.930751,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,51089,22.368270,14.0,3.070415,0.319071,1.597734,16.498246,6.0,6.140830,0.415334,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
order_info_summary.fillna(0.0,inplace=True)
order_info_summary.fillna(0,inplace=True)
order_info_summary.to_csv("../feature/mid/order_info_summary.csv",index=None,encoding='utf-8')